In [1]:
import sys
sys.path.append('../')

In [47]:
import pandas as pd
import numpy as np
from constants import *

In [83]:
data = pd.read_csv(f'{data_bihar}/Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.csv')
cols = list(data.columns.values)

/tmp/ipykernel_4137726/4201233215.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,2

### Timestamp, District, Block, Latitude and Longitude Information

In [88]:
def check(cols):
    if len(cols) % 3 != 0:
        return False
    
    for i in range(0, len(cols), 3):
        if cols[i][5:] != cols[i+1][5:] != cols[i+2][3:]:
            return False
    
    return True

def get_ids(col_names):
    cols = []
    for i in range(0, len(col_names), 3):
        cols.append(col_names[i].replace('PM25_', '').strip())
    return cols

In [93]:
device_ids = get_ids(cols[1:])
# Keep a look at the first index of ts, the first value might be present at a different row
ts = data.loc[6:, cols[0]].to_list()
districts = data.loc[0, cols[1]:].to_list()
blocks = data.loc[1, cols[1]:].to_list()
latitude = data.loc[2, cols[1]:].to_list()
longitude = data.loc[3, cols[1]:].to_list()

assert check(cols[1:]), "Problem in the csv file"
assert len(device_ids) * 3 == len(districts) == len(blocks) == len(latitude) == len(longitude), "Incorrect Data Loaded"
# print(device_ids, ts, districts, blocks, latitude, longitude)

### DataFrame initialization

In [110]:
def convert_to_df(data):

    df_cols = {'timestamp': np.datetime64, 'device_id': str, 'block': str, 'district': str, 'latitude': float, 'longitude': float,
            'rh': float, 'temp': float, 'pm25': float}
    df = pd.DataFrame(columns=[i for i in df_cols.keys()])
    df = df.astype(df_cols)
    df.dtypes

    for i in range(1, len(data.columns.values), 3):
        
        new_df = pd.DataFrame(columns=list(df.columns.values))
        new_df = new_df.astype(df_cols)

        # Adding new data row wise
        new_df['device_id'], new_df['district'], new_df['block'], new_df['latitude'], new_df['longitude'] =  device_ids[(i-1)//3], districts[i], blocks[i], latitude[i], longitude[i]
        new_df['timestamp'] = ts
        new_df['pm25'], new_df['temp'], new_df['rh'] = data.loc[6:, cols[i]], data.loc[6:, cols[i+1]], data.loc[6:, cols[i+2]]

        df = pd.concat([df, new_df])

    return df

In [111]:
data

,Deviceid,PM25_1201230001,Temp_1201230001,RH_1201230001,PM25_1201230003,Temp_1201230003,RH_1201230003,PM25_1201230004,Temp_1201230004,RH_1201230004,...,RH_FCF5C490265D,PM25_FCF5C4960193,Temp_FCF5C4960193,RH_FCF5C4960193,PM25_FCF5C4962BC1,Temp_FCF5C4962BC1,RH_FCF5C4962BC1,PM25_FCF5C4A1321B,Temp_FCF5C4A1321B,RH_FCF5C4A1321B
0,District,BUXAR,BUXAR,BUXAR,DARBHANGA,DARBHANGA,DARBHANGA,JAMUI,JAMUI,JAMUI,...,SAMASTIPUR,SITAMARHI,SITAMARHI,SITAMARHI,VAISHALI,VAISHALI,VAISHALI,SAMASTIPUR,SAMASTIPUR,SAMASTIPUR
1,Block,RAJPUR.B,RAJPUR.B,RAJPUR.B,KUSHESHWAR ASTHAN EAST,KUSHESHWAR ASTHAN EAST,KUSHESHWAR ASTHAN EAST,ISLAMNAGAR ALIGANJ,ISLAMNAGAR ALIGANJ,ISLAMNAGAR ALIGANJ,...,MOHIUDDINAGAR,BATHNAHA,BATHNAHA,BATHNAHA,LALGANJ,LALGANJ,LALGANJ,UJIARPUR,UJIARPUR,UJIARPUR
2,latitude,25.392,25.392,25.392,25.812,25.812,25.812,24.949,24.949,24.949,...,25.6,26.63331032,26.63331032,26.63331032,25.85486031,25.85486031,25.85486031,25.77,25.77,25.77
3,longitude,83.903,83.903,83.903,86.324,86.324,86.324,85.928,85.928,85.928,...,85.702,85.57259369,85.57259369,85.57259369,85.17176056,85.17176056,85.17176056,85.797,85.797,85.797
4,Remark,NILL,NILL,NILL,NILL,NILL,NILL,NILL,NILL,NILL,...,NILL,NILL,NILL,NILL,NILL,NILL,NILL,NILL,NILL,NILL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,2023-08-31 19:00:00,39.024713,38.620833,60.690517,19.420402,33.119828,68.348563,41.6,34.79,61.458333,...,98.971667,81.8,35.576667,70.838333,35.880769,36.930769,61.484744,NaN,33.670769,72.263846
2954,2023-08-31 20:00:00,45.5,38.611667,60.758333,23.891667,32.411667,73.708333,53.202941,34.225,63.594608,...,100.0,NaN,34.88,72.983333,38.3,36.035,67.068333,NaN,33.375,75.276667
2955,2023-08-31 21:00:00,NaN,37.85,63.816667,24.533333,32.24,75.216667,52.361782,34.324167,64.279023,...,100.0,NaN,34.410641,78.457949,48.2,35.328333,68.023333,NaN,33.026667,77.23
2956,2023-08-31 22:00:00,NaN,37.7,65.258333,24.206609,31.603276,75.204023,43.725,34.588333,63.766667,...,100.0,NaN,33.975,81.973333,55.329487,34.833974,72.896026,NaN,32.780897,79.559872


In [112]:
df = convert_to_df(data)
df

In [16]:
df.to_pickle(f'{data_bihar}/Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.pkl')